In [1]:
import numpy as np
import pandas as pd
import math
import netCDF4 as nc

Reading all nc files using netCDF4 library.

In [2]:
longwavefluxpath='C:/Users/gurur/Downloads/gefs_train/train/dlwrf_sfc_latlon_subset_19940101_20071231.nc'
longwavefluxfile=nc.Dataset(longwavefluxpath,'r')
shortwavefluxpath='C:/Users/gurur/Downloads/gefs_train/train/dswrf_sfc_latlon_subset_19940101_20071231.nc'
shortwavefluxfile=nc.Dataset(shortwavefluxpath,'r')
airpressurepath='C:/Users/gurur/Downloads/gefs_train/train/pres_msl_latlon_subset_19940101_20071231.nc'
airpressurefile=nc.Dataset(airpressurepath,'r')
precipwaterpath='C:/Users/gurur/Downloads/gefs_train/train/pwat_eatm_latlon_subset_19940101_20071231.nc'
precipwaterfile=nc.Dataset(precipwaterpath,'r')
humiditypath='C:/Users/gurur/Downloads/gefs_train/train/spfh_2m_latlon_subset_19940101_20071231.nc'
humidityfile=nc.Dataset(humiditypath,'r')
cloudpath='C:/Users/gurur/Downloads/gefs_train/train/tcdc_eatm_latlon_subset_19940101_20071231.nc'
cloudfile=nc.Dataset(cloudpath,'r')
temppath='C:/Users/gurur/Downloads/gefs_train/train/tmp_sfc_latlon_subset_19940101_20071231.nc'
tempfile=nc.Dataset(temppath,'r')
longwaveradpath='C:/Users/gurur/Downloads/gefs_train/train/ulwrf_sfc_latlon_subset_19940101_20071231.nc'
longwaveradfile=nc.Dataset(longwaveradpath,'r')
longwaveradApath='C:/Users/gurur/Downloads/gefs_train/train/ulwrf_tatm_latlon_subset_19940101_20071231.nc'
longwaveradAfile=nc.Dataset(longwaveradApath,'r')
shortwaveradpath='C:/Users/gurur/Downloads/gefs_train/train/uswrf_sfc_latlon_subset_19940101_20071231.nc'
shortwaveradfile=nc.Dataset(shortwaveradpath,'r')
path='C:/Users/gurur/Downloads/gefs_train/train/apcp_sfc_latlon_subset_19940101_20071231.nc'
ncfile=nc.Dataset(path,mode='r')
maxtemppath='C:/Users/gurur/Downloads/gefs_train/train/tmax_2m_latlon_subset_19940101_20071231.nc'
maxtempfile=nc.Dataset(maxtemppath,'r')

In [3]:
maxtempfile.variables

{'time': <class 'netCDF4._netCDF4.Variable'>
 float64 time(time)
     long_name: Time
     units: hours since 1800-01-01 00:00:00
     axis: T
 unlimited dimensions: 
 current shape = (5113,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'intTime': <class 'netCDF4._netCDF4.Variable'>
 int32 intTime(time)
     long_name: time as an integer (YYYYMMDDHH)
 unlimited dimensions: 
 current shape = (5113,)
 filling on, default _FillValue of -2147483647 used,
 'lat': <class 'netCDF4._netCDF4.Variable'>
 float32 lat(lat)
     long_name: Latitude
     standard_name: latitude
     units: degrees_north
     actual_range: [31. 39.]
     axis: Y
 unlimited dimensions: 
 current shape = (9,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'lon': <class 'netCDF4._netCDF4.Variable'>
 float32 lon(lon)
     long_name: Longitude
     standard_name: longitude
     units: degrees_east
     actual_range: [254. 269.]
     axis: X
 unlimited dimensions: 
 current shape = (16,)

Extracting features from nc file object.

In [4]:
longwaveflux=longwavefluxfile.variables['Downward_Long-Wave_Rad_Flux'][:]
shortwaveflux=shortwavefluxfile.variables['Downward_Short-Wave_Rad_Flux'][:]
airpressure=airpressurefile.variables['Pressure'][:]
precipwater=precipwaterfile.variables['Precipitable_water'][:]
humidity=humidityfile.variables['Specific_humidity_height_above_ground'][:]
cloudcover=cloudfile.variables['Total_cloud_cover'][:]
temp=tempfile.variables['Temperature_surface'][:]
longwaverad=longwaveradfile.variables['Upward_Long-Wave_Rad_Flux_surface'][:]
longwaveradA=longwaveradAfile.variables['Upward_Long-Wave_Rad_Flux'][:]
shortwaverad=shortwaveradfile.variables['Upward_Short-Wave_Rad_Flux'][:]
maxtemp=maxtempfile.variables['Maximum_temperature'][:]
precips=ncfile.variables['Total_precipitation'][:]

Extracting latitudes and longitudes of GEFS points

In [5]:
lats=ncfile.variables['lat'][:]
lons=ncfile.variables['lon'][:]

len(lats)

In [9]:
lats=ncfile.variables['lat'][:]
lats

masked_array(data=[31., 32., 33., 34., 35., 36., 37., 38., 39.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

Reading train.csv

In [6]:
traindf=pd.read_csv('C:/Users/gurur/Downloads/train.csv')
traindf.head()

,Date,ACME,ADAX,ALTU,APAC,ARNE,BEAV,BESS,BIXB,BLAC,...,VINI,WASH,WATO,WAUR,WEAT,WEST,WILB,WIST,WOOD,WYNO
0,19940101,12384900,11930700,12116700,12301200,10706100,10116900,11487900,11182800,10848300,...,10771800,12116400,11308800,12361800,11331600,10644300,11715600,11241000,10490100,10545300
1,19940102,11908500,9778500,10862700,11666400,8062500,9262800,9235200,3963300,3318300,...,4314300,10733400,9154800,12041400,9168300,4082700,9228000,5829900,7412100,3345300
2,19940103,12470700,9771900,12627300,12782700,11618400,10789800,11895900,4512600,5266500,...,2976900,11775000,10700400,12687300,11324400,2746500,3686700,4488900,9712200,4442100
3,19940104,12725400,6466800,13065300,12817500,12134400,11816700,12186600,3212700,8270100,...,3476400,12159600,11907000,12953100,11903700,2741400,4905000,4089300,11401500,4365000
4,19940105,10894800,11545200,8060400,10379400,6918600,9936300,6411300,9566100,8009400,...,6393300,11419500,7334400,10178700,7471500,8235300,11159100,10651500,10006200,8568300


No missing values in train.csv

In [7]:
traindf.isna().sum()

Date    0
ACME    0
ADAX    0
ALTU    0
APAC    0
       ..
WEST    0
WILB    0
WIST    0
WOOD    0
WYNO    0
Length: 99, dtype: int64

Reading station_info.csv

In [7]:
stationdf=pd.read_csv('C:/Users/gurur/Downloads/station_info.csv')
stationdf.head()

,stid,nlat,elon,elev
0,ACME,34.80833,-98.02325,397
1,ADAX,34.79851,-96.66909,295
2,ALTU,34.58722,-99.33808,416
3,APAC,34.91418,-98.29216,440
4,ARNE,36.07204,-99.90308,719


In [18]:
s=stationdf[stationdf['stid']=='ACME']
s.elev[0]

397

Function for calculating haversine distance between 2 points using their latitude and longitude.

In [8]:
def dist(lat1,lon1,lat2,lon2):
    dlat=math.radians(lat1)-math.radians(lat2)
    dlon=math.radians(lon1)-math.radians(lon2)
    a=math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    c=2*math.atan2(math.sqrt(a),math.sqrt(1-a))
    d=6373.0*c
    if d<=200:
        return True
    return False

Combining data from nc files, train.csv, station_info.csv into one dataframe

In [9]:
#cols=traindf.columns
cols=['HINT','IDAB','SLAP','WEST','BESS','ACME']
itervar=0
new_df=pd.DataFrame(columns=['Date','Month','station','station elevation','Precipitation','Downward Long Wave Rad Flux','Downward Short Wave Rad Flux','Air Pressure','Water precipitation','Humidity','Cloud cover','Temperature','Upward Long wave Rad','Upward Long wave Rad Atmos','Upward Short Wave Rad','Max Temperature','solar energy'])
for index,row in traindf.iterrows():
    date=row.Date
    for i in range(len(cols)):
        st=stationdf[stationdf['stid']==cols[i]]
        date=str(date)
        newdate=date[:4] + '-' + date[4:6] + '-' + date[6:]
        month=date[4:6]
        totprecip=0.0
        totlongwaveflux=0.0
        totshortwaveflux=0.0
        totairpressure=0.0
        totprecipwater=0.0
        tothumidity=0.0
        totcloudcover=0.0
        tottemp=0.0
        totlongwaverad=0.0
        totlongwaveradA=0.0
        totshortwaverad=0.0
        totmaxtemp=0.0
        cntcloud=0
        cnttemp=0
        cntmaxtemp=0
        
        for m in range(len(lats)):
            for n in range(len(lons)):
                if dist(lats[m],lons[n],st.nlat,st.elon+360):
                    totprecip=totprecip+np.sum(precips[index,:,:,m,n])
                    totlongwaveflux=totlongwaveflux+np.sum(longwaveflux[index,:,:,m,n])
                    totshortwaveflux=totshortwaveflux+np.sum(shortwaveflux[index,:,:,m,n])
                    totairpressure=totairpressure+np.sum(airpressure[index,:,:,m,n])
                    totprecipwater=totprecipwater+np.sum(precipwater[index,:,:,m,n])
                    tothumidity=tothumidity+np.sum(humidity[index,:,:,m,n])
                    totcloudcover=totcloudcover+np.sum(cloudcover[index,:,:,m,n])
                    tottemp=tottemp+np.sum(temp[index,:,:,m,n])
                    totlongwaverad=totlongwaverad+np.sum(longwaverad[index,:,:,m,n])
                    totlongwaveradA=totlongwaveradA+np.sum(longwaveradA[index,:,:,m,n])
                    totshortwaverad=totshortwaverad+np.sum(shortwaverad[index,:,:,m,n])
                    totmaxtemp=totmaxtemp+np.sum(maxtemp[index,:,:,m,n])
                    cntcloud=cntcloud+1
                    cnttemp=cnttemp+1
                    cntmaxtemp=cntmaxtemp+1
                    
        if cntcloud!=0:
            totcloudcover=totcloudcover/cntcloud
        if cnttemp!=0:
            tottemp=tottemp/cnttemp
        if cntmaxtemp!=0:
            totmaxtemp=totmaxtemp/cntmaxtemp
        
        elev=st['elev'].values[0]
        new_dict={'Date':newdate,'Month':month,'station':cols[i],'station elevation':elev,'Precipitation':totprecip,'Downward Long Wave Rad Flux':totlongwaveflux,'Downward Short Wave Rad Flux':totshortwaveflux,'Air Pressure':totairpressure,'Water precipitation':totprecipwater,'Humidity':tothumidity,'Cloud cover':totcloudcover,'Temperature':tottemp,'Upward Long wave Rad':totlongwaverad,'Upward Long wave Rad Atmos':totlongwaveradA,'Upward Short Wave Rad':totshortwaverad,'Max Temperature':totmaxtemp,'solar energy':row[cols[i]]}
        new_df=new_df.append(new_dict,ignore_index=True)
        itervar=itervar+1
        if itervar%100000==0:
            print("Number of records populated:",itervar)

In [10]:
new_df.head()

,Date,Month,station,station elevation,Precipitation,Downward Long Wave Rad Flux,Downward Short Wave Rad Flux,Air Pressure,Water precipitation,Humidity,Cloud cover,Temperature,Upward Long wave Rad,Upward Long wave Rad Atmos,Upward Short Wave Rad,Max Temperature,solar energy
0,1994-01-01,01,HINT,493,0.000000,168994.609375,141390.0,67344117.5,5479.057648,2.158353,0.085000,15510.469076,238776.054688,166465.324219,31795.0,15605.287272,11604900
1,1994-01-01,01,IDAB,110,133.729992,192001.623047,137450.0,67270510.5,7606.557434,3.417633,2.556667,15550.102132,239394.054688,162546.324219,25254.0,15614.737549,8080500
2,1994-01-01,01,SLAP,774,0.000000,175262.910156,147070.0,72959447.0,5550.804291,1.953540,0.216923,15431.745868,252431.642578,175520.434570,34294.0,15491.706656,10265700
3,1994-01-01,01,WEST,348,12.010000,175368.609375,133060.0,67224738.5,6271.757782,2.755343,0.615833,15367.985433,228877.054688,157144.324219,26398.0,15435.445394,10644300
4,1994-01-01,01,BESS,511,0.000000,210058.511719,178190.0,84188831.5,6624.797089,2.527614,0.102000,15524.815755,299059.818359,208039.655273,40640.0,15603.784049,11487900


In [12]:
new_df

,Date,Month,station,station elevation,Precipitation,Downward Long Wave Rad Flux,Downward Short Wave Rad Flux,Air Pressure,Water precipitation,Humidity,Cloud cover,Temperature,Upward Long wave Rad,Upward Long wave Rad Atmos,Upward Short Wave Rad,Max Temperature,solar energy
0,1994-01-01,01,HINT,493,0.000000,112359.406250,94540.0,44895861.5,3608.671814,1.405125,0.090000,15509.144043,158947.703125,110788.882812,21442.0,15602.162598,11604900
1,1994-01-01,01,IDAB,110,60.979995,112622.113281,83110.0,39242359.0,4436.125183,2.047889,2.210000,15583.654436,140809.115234,95704.772461,14938.0,15641.623326,8080500
2,1994-01-01,01,SLAP,774,0.000000,81510.804688,68140.0,33677743.0,2601.378906,0.914512,0.190000,15442.669108,117093.527344,81536.662109,15584.0,15511.304036,10265700
3,1994-01-01,01,WEST,348,5.940000,116251.406250,89360.0,44817132.0,4135.571869,1.846015,0.418750,15362.081299,152321.703125,104797.882812,17514.0,15427.862183,10644300
4,1994-01-01,01,BESS,511,0.000000,83879.804688,71000.0,33677687.5,2672.178894,1.015642,0.100000,15508.618815,119316.527344,83322.662109,16282.0,15607.704427,11487900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24103,2004-12-31,12,IDAB,110,112.559997,145718.164062,36880.0,39271663.5,10143.052979,4.305173,6.334286,15995.818080,155728.140625,98027.842773,5702.0,16026.319475,4485300
24104,2004-12-31,12,SLAP,774,0.000000,88276.123047,66350.0,33493886.5,2616.830872,1.433677,0.755000,15677.041341,122994.406250,81788.867188,14952.0,15766.227376,9441000
24105,2004-12-31,12,WEST,348,71.970002,156914.181641,46730.0,44825480.5,8793.174744,4.446489,3.985000,15892.875488,173588.875000,118821.825195,8185.0,15938.485229,5932800
24106,2004-12-31,12,BESS,511,0.000000,95309.122070,68990.0,33518187.5,3699.430847,2.072987,0.328333,15809.074544,127057.406250,87493.871094,15548.0,15896.750977,10535400


In [11]:
new_df.to_csv('trainData6stationsWithMon200.csv')

In [12]:
traindf.columns

Index(['Date', 'ACME', 'ADAX', 'ALTU', 'APAC', 'ARNE', 'BEAV', 'BESS', 'BIXB',
       'BLAC', 'BOIS', 'BOWL', 'BREC', 'BRIS', 'BUFF', 'BURB', 'BURN', 'BUTL',
       'BYAR', 'CAMA', 'CENT', 'CHAN', 'CHER', 'CHEY', 'CHIC', 'CLAY', 'CLOU',
       'COOK', 'COPA', 'DURA', 'ELRE', 'ERIC', 'EUFA', 'FAIR', 'FORA', 'FREE',
       'FTCB', 'GOOD', 'GUTH', 'HASK', 'HINT', 'HOBA', 'HOLL', 'HOOK', 'HUGO',
       'IDAB', 'JAYX', 'KENT', 'KETC', 'LAHO', 'LANE', 'MADI', 'MANG', 'MARE',
       'MAYR', 'MCAL', 'MEDF', 'MEDI', 'MIAM', 'MINC', 'MTHE', 'NEWK', 'NINN',
       'NOWA', 'OILT', 'OKEM', 'OKMU', 'PAUL', 'PAWN', 'PERK', 'PRYO', 'PUTN',
       'REDR', 'RETR', 'RING', 'SALL', 'SEIL', 'SHAW', 'SKIA', 'SLAP', 'SPEN',
       'STIG', 'STIL', 'STUA', 'SULP', 'TAHL', 'TALI', 'TIPT', 'TISH', 'VINI',
       'WASH', 'WATO', 'WAUR', 'WEAT', 'WEST', 'WILB', 'WIST', 'WOOD', 'WYNO'],
      dtype='object')

mask_dict.keys

In [33]:
traindf

,Date,ACME,ADAX,ALTU,APAC,ARNE,BEAV,BESS,BIXB,BLAC,...,VINI,WASH,WATO,WAUR,WEAT,WEST,WILB,WIST,WOOD,WYNO
0,19940101,12384900,11930700,12116700,12301200,10706100,10116900,11487900,11182800,10848300,...,10771800,12116400,11308800,12361800,11331600,10644300,11715600,11241000,10490100,10545300
1,19940102,11908500,9778500,10862700,11666400,8062500,9262800,9235200,3963300,3318300,...,4314300,10733400,9154800,12041400,9168300,4082700,9228000,5829900,7412100,3345300
2,19940103,12470700,9771900,12627300,12782700,11618400,10789800,11895900,4512600,5266500,...,2976900,11775000,10700400,12687300,11324400,2746500,3686700,4488900,9712200,4442100
3,19940104,12725400,6466800,13065300,12817500,12134400,11816700,12186600,3212700,8270100,...,3476400,12159600,11907000,12953100,11903700,2741400,4905000,4089300,11401500,4365000
4,19940105,10894800,11545200,8060400,10379400,6918600,9936300,6411300,9566100,8009400,...,6393300,11419500,7334400,10178700,7471500,8235300,11159100,10651500,10006200,8568300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4013,20041227,7236300,7627800,9201000,8952600,8961600,8672100,10174200,8080800,7129800,...,8152200,7782900,9324300,8893500,8951700,9663300,8505300,9018300,9350400,8458500
4014,20041228,9362700,5974500,11710800,9124500,11166000,10980300,11340300,7657200,10353300,...,7920300,6183300,10903800,7215300,11086500,7700400,11555400,10745400,10605600,8035200
4015,20041229,6981000,6741300,6506100,6564300,7060800,9898500,6695400,6620400,7662000,...,4512000,7089000,7722300,7380000,7250400,2478600,2223900,3264600,7832700,8811900
4016,20041230,7562100,6176100,10876500,9515700,10974900,10783500,11021100,7700400,9975300,...,4683300,7330800,10586100,7484700,10934700,3936300,5303100,4982100,10936200,8181900


In [12]:
testdf=pd.read_csv('C:/Users/gurur/Desktop/Term1 Docs/ML/MLProject/test.csv')
testdf.head()

,Date
0,HINT-20050101
1,HINT-20050102
2,HINT-20050103
3,HINT-20050104
4,HINT-20050105


In [13]:
new_testdf=pd.DataFrame(columns=['Date','Month','station','station elevation','Precipitation','Downward Long Wave Rad Flux','Downward Short Wave Rad Flux','Air Pressure','Water precipitation','Humidity','Cloud cover','Temperature','Upward Long wave Rad','Upward Long wave Rad Atmos','Upward Short Wave Rad','Max Temperature'])
index=4018
for ind,row in testdf.iterrows():
    if index==5113:
        index=4018
    r=str(row['Date'])
    station=r.split('-')
    date=station[1]
    date=str(date)
    month=date[4:6]
    station=station[0]
    st=stationdf[stationdf['stid']==station]
    totprecip=0.0
    totlongwaveflux=0.0
    totshortwaveflux=0.0
    totairpressure=0.0
    totprecipwater=0.0
    tothumidity=0.0
    totcloudcover=0.0
    tottemp=0.0
    totlongwaverad=0.0
    totlongwaveradA=0.0
    totshortwaverad=0.0
    totmaxtemp=0.0
    cntcloud=0
    cnttemp=0
    cntmaxtemp=0
        
    for m in range(len(lats)):
        for n in range(len(lons)):
            if dist(lats[m],lons[n],st.nlat,st.elon+360):
                totprecip=totprecip+np.sum(precips[index,:,:,m,n])
                totlongwaveflux=totlongwaveflux+np.sum(longwaveflux[index,:,:,m,n])
                totshortwaveflux=totshortwaveflux+np.sum(shortwaveflux[index,:,:,m,n])
                totairpressure=totairpressure+np.sum(airpressure[index,:,:,m,n])
                totprecipwater=totprecipwater+np.sum(precipwater[index,:,:,m,n])
                tothumidity=tothumidity+np.sum(humidity[index,:,:,m,n])
                totcloudcover=totcloudcover+np.sum(cloudcover[index,:,:,m,n])
                tottemp=tottemp+np.sum(temp[index,:,:,m,n])
                totlongwaverad=totlongwaverad+np.sum(longwaverad[index,:,:,m,n])
                totlongwaveradA=totlongwaveradA+np.sum(longwaveradA[index,:,:,m,n])
                totshortwaverad=totshortwaverad+np.sum(shortwaverad[index,:,:,m,n])
                totmaxtemp=totmaxtemp+np.sum(maxtemp[index,:,:,m,n])
                cntcloud=cntcloud+1
                cnttemp=cnttemp+1
                cntmaxtemp=cntmaxtemp+1
                    
    if cntcloud!=0:
        totcloudcover=totcloudcover/cntcloud
    if cnttemp!=0:
        tottemp=tottemp/cnttemp
    if cntmaxtemp!=0:
        totmaxtemp=totmaxtemp/cntmaxtemp
        
    elev=st['elev'].values[0]
    new_dict={'Date':date,'Month':month,'station':station,'station elevation':elev,'Precipitation':totprecip,'Downward Long Wave Rad Flux':totlongwaveflux,'Downward Short Wave Rad Flux':totshortwaveflux,'Air Pressure':totairpressure,'Water precipitation':totprecipwater,'Humidity':tothumidity,'Cloud cover':totcloudcover,'Temperature':tottemp,'Upward Long wave Rad':totlongwaverad,'Upward Long wave Rad Atmos':totlongwaveradA,'Upward Short Wave Rad':totshortwaverad,'Max Temperature':totmaxtemp}
    new_testdf=new_testdf.append(new_dict,ignore_index=True)
    index=index+1

In [14]:
new_testdf

,Date,Month,station,station elevation,Precipitation,Downward Long Wave Rad Flux,Downward Short Wave Rad Flux,Air Pressure,Water precipitation,Humidity,Cloud cover,Temperature,Upward Long wave Rad,Upward Long wave Rad Atmos,Upward Short Wave Rad,Max Temperature
0,20050101,01,HINT,493,122.120000,232655.207031,48010.0,67069826.0,14283.812439,6.377589,6.549167,15862.920654,257883.007812,146324.938477,8775.0,15903.641764
1,20050102,01,HINT,493,205.790000,235791.162109,55910.0,67298614.5,17168.335083,6.394114,15.099166,15864.267171,259417.984375,121673.308594,9278.0,15902.177002
2,20050103,01,HINT,493,740.199986,231617.289062,82970.0,67348028.5,16797.600342,5.546970,7.055000,15768.031576,253955.460938,154372.404297,16153.0,15767.199707
3,20050104,01,HINT,493,2080.179993,228650.195312,32798.0,67175896.0,21019.107544,4.521409,26.138334,15482.608154,232968.312500,125524.590820,6223.0,15497.127035
4,20050105,01,HINT,493,1793.990089,186375.539062,97000.0,67590169.5,8579.307373,1.496333,12.545000,14786.329834,199873.854492,111598.304688,44597.0,14864.711426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,20071227,12,ACME,397,31.029999,148159.857422,111700.0,61440467.0,4976.737091,2.068159,1.603636,15100.919300,192766.064453,143812.496094,33765.0,15099.346857
6566,20071228,12,ACME,397,18.240000,138764.490234,133480.0,61803143.5,3673.335480,1.780084,0.857273,15026.739702,191794.939453,142885.294922,48273.0,15061.697621
6567,20071229,12,ACME,397,0.220000,139906.806641,128610.0,61770160.0,4853.691254,1.726038,1.883636,15038.640891,190038.519531,137495.267578,47456.0,15094.801669
6568,20071230,12,ACME,397,0.010000,137877.939453,137620.0,61552529.0,2820.396011,1.863900,0.003636,15138.685458,195798.931641,147705.246094,47244.0,15240.020774


In [15]:
new_testdf.to_csv('testdfWithMon200.csv')